### 11.8 LSTM
論文データを学習させてナンチャッテ論文をlatex形式で出力させる<br>
LSTM版

### ライブラリインポート

In [1]:
!pip install datasets tqdm torch requests feedparser --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 2.9 MB/s eta 0:00:00


In [2]:
import os
import tarfile
import glob
import re
import requests
from tqdm import tqdm
import urllib.parse
import feedparser
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

### データセット読み込み

#### arXiv API から論文 ID をまとめて取得する関数

In [3]:
def fetch_arxiv_ids(category="cs.LG", max_results=200):
    """
    指定カテゴリの新しい arXiv 論文 ID をまとめて取得する簡易関数
    例: category="cs.LG"（Machine Learning）
    """
    base_url = "http://export.arxiv.org/api/query"

    # APIのクエリ: カテゴリ指定 + 新しい順
    search_query = f"cat:{category}"
    params = {
        "search_query": search_query,
        "start": 0,
        "max_results": max_results,
        "sortBy": "submittedDate",
        "sortOrder": "descending",
    }

    url = base_url + "?" + urllib.parse.urlencode(params)
    print("🔍 arXiv API URL:", url)

    feed = feedparser.parse(url)

    paper_ids = []
    for entry in feed.entries:
        # entry.id 例: "http://arxiv.org/abs/2401.12345v1"
        m = re.search(r'arxiv.org/abs/(\d{4}\.\d+)', entry.id)
        if m:
            pid = m.group(1)  # "2401.12345"
            paper_ids.append(pid)

    print(f"📄 取得した論文ID数: {len(paper_ids)} 件")
    return paper_ids

# ここで好きなカテゴリ・件数を指定
# 例1: cs.LG (Machine Learning) 最新から200本
# 例2: math.PR (Probability)
paper_ids = fetch_arxiv_ids(category="cs.LG", max_results=200)

print(paper_ids)

🔍 arXiv API URL: http://export.arxiv.org/api/query?search_query=cat%3Acs.LG&start=0&max_results=200&sortBy=submittedDate&sortOrder=descending
📄 取得した論文ID数: 200 件
['2511.21690', '2511.21689', '2511.21686', '2511.21678', '2511.21675', '2511.21669', '2511.21668', '2511.21667', '2511.21654', '2511.21652', '2511.21638', '2511.21635', '2511.21626', '2511.21622', '2511.21613', '2511.21607', '2511.21600', '2511.21594', '2511.21590', '2511.21581', '2511.21566', '2511.21561', '2511.21560', '2511.21550', '2511.21537', '2511.21531', '2511.21526', '2511.21514', '2511.21513', '2511.21500', '2511.21490', '2511.21474', '2511.21466', '2511.21465', '2511.21437', '2511.21416', '2511.21414', '2511.21408', '2511.21397', '2511.21381', '2511.21378', '2511.21377', '2511.21369', '2511.21364', '2511.21363', '2511.21356', '2511.21354', '2511.21350', '2511.21340', '2511.21338', '2511.21335', '2511.21320', '2511.21283', '2511.21276', '2511.21257', '2511.21247', '2511.21232', '2511.21223', '2511.21215', '2511.21213'

#### arXiv ソースをダウンロード＆展開

In [4]:
os.makedirs("papers", exist_ok=True)

for pid in tqdm(paper_ids, desc="Downloading", ncols=80, ascii=True):
    url = f"https://arxiv.org/src/{pid}v1"
    out_path = f"papers/{pid}.tar.gz"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            with open(out_path, "wb") as f:
                f.write(r.content)
            # tar / tar.gz / その他の圧縮形式に柔軟に対応
            try:
                with tarfile.open(out_path, "r:*") as tar:
                    tar.extractall(f"papers/{pid}")
            except tarfile.ReadError:
                print(f"Skipped {pid} (not a tar file)")
        else:
            print(f"Skipped {pid} (status {r.status_code})")
    except Exception as e:
        print(f"Error downloading {pid}: {e}")


Downloading:   0%|                                      | 0/200 [00:00<?, ?it/s]/tmp/ipython-input-2326405988.py:14: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(f"papers/{pid}")
Downloading:  11%|###1                         | 22/200 [00:33<02:14,  1.33it/s]

Skipped 2511.21561 (not a tar file)


Downloading:  38%|##########8                  | 75/200 [02:04<00:56,  2.21it/s]

Skipped 2511.21089 (not a tar file)


Downloading:  44%|############9                | 89/200 [03:27<25:21, 13.71s/it]

...

Downloading:  60%|################6           | 119/200 [04:16<01:28,  1.10s/it]

Skipped 2511.20871 (not a tar file)


Downloading:  94%|##########################1 | 187/200 [10:27<00:20,  1.58s/it]

Skipped 2511.20382 (not a tar file)


Downloading: 100%|############################| 200/200 [10:52<00:00,  3.26s/it]


#### 結果確認

In [5]:
!find papers -name "*.tex" | head -n 20

papers/2511.20406/imports.tex
papers/2511.20406/log_2025.tex
papers/2511.21550/content/intro.tex
papers/2511.21550/content/heavy_momentum_mamba.tex
papers/2511.21550/content/related_work.tex
papers/2511.21550/content/_abstract.tex
papers/2511.21550/content/result.tex
papers/2511.21550/content/abstract.tex
papers/2511.21550/content/har_system.tex
papers/2511.21550/content/vanishing_gradient.tex
papers/2511.21550/content/intro_.tex
papers/2511.21550/content/prelim.tex
papers/2511.21550/content/conclusion.tex
papers/2511.21550/content/beyond.tex
papers/2511.21550/content/har_system_.tex
papers/2511.21550/content/prelim_.tex
papers/2511.21550/content/continuous_momentum.tex
papers/2511.21550/New_IEEEtran_how-to.tex
papers/2511.21550/bare_jrnl_new_sample4.tex
papers/2511.21550/math_commands.tex


### データ加工

#### LaTeX 本文抽出 ＋ 前処理

In [24]:
import re, glob

def extract_latex_body(tex_text: str) -> str:
    """LaTeX本文を抽出してある程度クリーンアップする（統一版）"""
    # コメント削除
    text = re.sub(r'%.*', '', tex_text)

    # \begin{document}〜\end{document} 抜き出し
    m = re.search(r'\\begin{document}(.*?)\\end{document}', text, re.DOTALL)
    if m:
        text = m.group(1)

    # bibliography / appendix 以降をざっくりカット
    text = re.split(r'\\bibliography|\\begin{thebibliography}', text)[0]
    text = re.split(r'\\appendix', text)[0]

    # 空白を1つに
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [25]:
all_texts = []
for fname in glob.glob("papers/**/*.tex", recursive=True):
    try:
        with open(fname, encoding="utf-8", errors="ignore") as f:
            raw = f.read()
        cleaned = extract_latex_body(raw)
        if len(cleaned) > 200:
            all_texts.append(cleaned)
    except Exception as e:
        print(f"{fname}: {e}")

print(f"抽出された論文本数: {len(all_texts)}")
print(f"総テキスト長: {sum(len(t) for t in all_texts):,} 文字")


抽出された論文本数: 860
総テキスト長: 11,596,237 文字


#### トークン化と語彙辞書作成

In [26]:
from collections import Counter

# SPECIAL_TOKENS はさっきと同じものを使う
# SPECIAL_TOKENS = ["<unk>", "<bos>", "<eos>"]

print(f"トークン総数: {len(tokens):,}")

# 語彙を頻度上位だけに圧縮
counter = Counter(tokens)

VOCAB_LIMIT = 40000
most_common_tokens = [
    w for (w, c) in counter.most_common(VOCAB_LIMIT)
    if w not in SPECIAL_TOKENS
]

vocab = SPECIAL_TOKENS + most_common_tokens

word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for i, w in enumerate(vocab)}

UNK_IDX = word2idx["<unk>"]
BOS_IDX = word2idx["<bos>"]
EOS_IDX = word2idx["<eos>"]

print(f"語彙数（圧縮後）: {len(vocab):,}")

# トークン列を index に変換
indexed_tokens = [word2idx.get(t, UNK_IDX) for t in tokens]

# 念のためチェック
max_id = max(indexed_tokens)
print("最大トークンID:", max_id)
print("語彙サイズ:", len(vocab))
assert max_id < len(vocab), "トークンIDが語彙サイズを超えています！"


トークン総数: 2,095,436
語彙数（圧縮後）: 40,001
最大トークンID: 40000
語彙サイズ: 40001


### データセット定義

#### Datasetクラス定義

In [27]:
class LatexDataset(Dataset):
    """大きなトークン列から、ランダムにサンプルを切り出す Dataset"""
    def __init__(self, data, seq_length=60, samples_per_epoch=3000):
        self.data = data                      # ← すでに index 化された列
        self.seq_length = seq_length
        self.samples_per_epoch = samples_per_epoch

        self.max_start = len(self.data) - (self.seq_length + 1)
        assert self.max_start > 0, "データが短すぎます"

    def __len__(self):
        # 1epochあたりのサンプル数を直接決める
        return self.samples_per_epoch

    def __getitem__(self, idx):
        # idx は使わず、毎回ランダムな位置から切り出す
        start = random.randint(0, self.max_start)
        end = start + self.seq_length + 1
        chunk = self.data[start:end]

        x = torch.tensor(chunk[:-1], dtype=torch.long)  # [T]
        y = torch.tensor(chunk[1:], dtype=torch.long)   # [T]
        return x, y


In [28]:
# ここで初めて dataset を作る（tokens ではなく indexed_tokens を渡す）
seq_length = 128
samples_per_epoch = 8000
dataset = LatexDataset(indexed_tokens, seq_length=seq_length,
                       samples_per_epoch=samples_per_epoch)


#### Train / Validation に分割して DataLoader を作成

In [29]:
train_ratio = 0.9
n_total = len(dataset)
n_train = int(n_total * train_ratio)
n_val = n_total - n_train

train_ds, val_ds = random_split(dataset, [n_train, n_val])

batch_size = 32

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=2,
    pin_memory=True
)

print(f"データ数: total={n_total}, train={len(train_ds)}, val={len(val_ds)}")
print(f"1epoch のステップ数（train）: {len(train_loader)}")


データ数: total=8000, train=7200, val=800
1epoch のステップ数（train）: 225


### 学習

#### LSTMモデル定義(疑似数学論文生成)

In [14]:
# LSTMモデル定義（疑似数学論文生成）
class LatexLSTM(nn.Module):
    """
    LaTeX トークン列用のシンプルな LSTM 言語モデル
    入力: x [B, T]  (トークンID)
    出力: logits [B, T, V]  (各位置ごとの単語分布のロジット)
    """
    def __init__(self,
                 vocab_size: int,
                 d_model: int = 384,
                 num_layers: int = 3,
                 dropout: float = 0.1):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.num_layers = num_layers
        # 語彙埋め込み
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        # LSTM
        # batch_first=True にして [B, T, E] 形式で扱えるようにする
        self.lstm = nn.LSTM(
            input_size=d_model,
            hidden_size=d_model,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        # 出力層
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: [B, T]  トークンID列
        戻り値:
            logits: [B, T, V]
        """
        # 埋め込み
        emb = self.token_embedding(x)   # [B, T, d_model]
        # LSTM に通す（隠れ状態は毎バッチごとにリセット）
        out, _ = self.lstm(emb)        # out: [B, T, d_model]
        # 各時刻ごとに語彙分布のロジットを計算
        logits = self.fc_out(out)      # [B, T, vocab_size]
        return logits


#### 繰り返し計算

In [15]:
%%time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

vocab_size = len(vocab)

model = LatexLSTM(
    vocab_size=vocab_size,
    d_model=384,      # 埋め込み・隠れ状態の次元
    num_layers=3,     # LSTMの層数（適宜調整可）
    dropout=0.1
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

num_epochs = 30
max_grad_norm = 1.0  # 勾配クリッピングの最大ノルム

for epoch in range(num_epochs):
    # -------------------------------
    # Train
    # -------------------------------
    model.train()
    total_train_loss = 0.0

    for x, y in train_loader:
        # x, y: [B, T]
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)  # [B, T, V]

        # CrossEntropyLoss は [N, C] vs [N] をとるので reshape する
        loss = criterion(
            logits.view(-1, logits.size(-1)),  # [B*T, V]
            y.view(-1)                         # [B*T]
        )
        loss.backward()

        # 勾配クリッピング（勾配爆発対策）
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # -------------------------------
    # Validation
    # -------------------------------
    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            val_loss = criterion(
                logits.view(-1, logits.size(-1)),
                y.view(-1)
            )
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_loader)

    # 学習率スケジューラを進める
    scheduler.step()

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"train_loss={avg_train_loss:.4f}  val_loss={avg_val_loss:.4f}  "
        f"lr={scheduler.get_last_lr()[0]:.5f}"
    )


device: cuda
Epoch [1/30] train_loss=8.0473  val_loss=7.8898  lr=0.00100
Epoch [2/30] train_loss=7.8975  val_loss=7.8932  lr=0.00100
Epoch [3/30] train_loss=7.8764  val_loss=7.8662  lr=0.00100
Epoch [4/30] train_loss=7.8755  val_loss=7.8656  lr=0.00100
Epoch [5/30] train_loss=7.8545  val_loss=7.8449  lr=0.00100
Epoch [6/30] train_loss=7.8381  val_loss=7.7268  lr=0.00100
Epoch [7/30] train_loss=7.4000  val_loss=7.0296  lr=0.00100
Epoch [8/30] train_loss=6.6720  val_loss=6.3780  lr=0.00100
Epoch [9/30] train_loss=6.2160  val_loss=6.0579  lr=0.00100
Epoch [10/30] train_loss=5.9441  val_loss=5.7028  lr=0.00050
Epoch [11/30] train_loss=5.7280  val_loss=5.6101  lr=0.00050
Epoch [12/30] train_loss=5.6257  val_loss=5.5276  lr=0.00050
Epoch [13/30] train_loss=5.5333  val_loss=5.3985  lr=0.00050
Epoch [14/30] train_loss=5.4601  val_loss=5.3182  lr=0.00050
Epoch [15/30] train_loss=5.3709  val_loss=5.2645  lr=0.00050
Epoch [16/30] train_loss=5.2839  val_loss=5.2196  lr=0.00050
Epoch [17/30] train_

### 予測

#### サンプリング用ヘルパー




In [16]:
def sample_from_logits(logits: torch.Tensor,
                       temperature: float = 0.8,
                       top_k: int = 50) -> int:
    """
    logits: [V] の1時刻分のロジット
    戻り値: 次に選ぶトークンID
    """
    # 温度でスケーリング
    logits = logits / temperature

    # top-k だけ残してサンプリング（極端なノイズを減らす）
    if top_k is not None and top_k > 0:
        values, indices = torch.topk(logits, top_k)
        probs = torch.softmax(values, dim=-1)
        idx_in_topk = torch.multinomial(probs, 1).item()
        next_token_id = indices[idx_in_topk].item()
        return next_token_id
    else:
        probs = torch.softmax(logits, dim=-1)
        next_token_id = torch.multinomial(probs, 1).item()
        return next_token_id


#### テキスト生成関数

In [17]:
def generate(seed_text, length=200, temperature=0.8, top_k=50, seq_length=60):
    model.eval()
    words = re.findall(TOKEN_PATTERN, seed_text)

    for _ in range(length):
        # 🔥 直近 seq_length 分だけ使う
        seq = [word2idx.get(w, UNK_IDX) for w in words[-seq_length:]]

        x = torch.tensor([seq], dtype=torch.long).to(device)

        with torch.no_grad():
            logits = model(x)[0, -1]  # ラスト時間ステップのみ

            # <unk> を出にくくする
            logits[UNK_IDX] -= 10.0

            logits = logits / temperature

            # top-k サンプリング（推奨）
            values, indices = torch.topk(logits, top_k)
            probs = torch.softmax(values, dim=0)
            next_idx = indices[torch.multinomial(probs, 1).item()].item()

        words.append(idx2word[next_idx])

    return " ".join(words)


#### テキスト生成サンプル

In [22]:
seed = r"""
\section{Mathematical Formulation}
We define the objective function as
\begin{equation}
"""

text = generate(seed, length=200, temperature=0.6, top_k=25, seq_length=seq_length)
print(text)


\section {Mathematical Formulation} We define the objective function as \begin {equation} \mathcal {L}_{ \text {SFT}} ( \theta }(s \mathcal {L}_{ \text {SFT}}( \theta ^{ \theta x^i y^i) \end {equation} where \mathcal {V} is the number of nodes is the set of a hidden distribution of the loss of the state space This is not a single estimate of the distribution of the input and the predicted embedding is then a small number of values in the first stage \begin {equation} \label {eq graph_obj} \mathbb {E} \left ( \mathbf {X}^ \top } \mathrm {cal}} \right ) \right ) = \hat { \mathbf {x}} + \mathbf {p} = \mathbf {x}) - \mathbf {W}_{ \mathrm {aug}}^{( \top \mathbf {I}) \end {equation} where \boldsymbol { \mathbf {x}} = \mathbf {b} \cdot \mathbf {x} ) We obtain the following set of the number of concepts in the current model we can have \begin {align} \mathcal {L}( \theta ) = \frac { \mathcal {L}_{ \text {in} \phi ( \theta _ \text {d}^2 \sigma _{ \text {dq} \W _ \text {rope} \mathbf {R}^ \mathb

#### 整形
上の結果は生成AIで整形をかけます。その結果を下のnotebookのr"""から"""の間に貼り付けます

#### PDF生成

In [23]:
# =====================================================
# LaTeX → PDF 変換 (Google Colab / Linux 環境用)
# =====================================================

import os, subprocess

# ① 生成済みLaTeX本文をここに貼る（整形後のtextを入れる）
latex_code = r"""
\documentclass{article}
\usepackage[T1]{fontenc}
\usepackage{lmodern}
\usepackage{amsmath, amssymb}
\usepackage{bm}

\begin{document}

\section{Mathematical Formulation}

We define the overall training objective in terms of a supervised fine-tuning (SFT) loss over the model parameters $\theta$. Given an input token sequence $\bm{x}^i$ and the corresponding target $\bm{y}^i$, the loss function can be written as

\begin{equation}
    \mathcal{L}_{\mathrm{SFT}}(\theta)
    =
    - \sum_{i=1}^{N}
    \log p_{\theta}(\bm{y}^i \mid \bm{x}^i),
    \label{eq:sft}
\end{equation}
where $N$ is the number of training samples, and $p_{\theta}$ denotes the conditional probability defined by the model.

In order to incorporate latent structure, we consider an embedding $\bm{z}$ derived from $\bm{x}$, and reformulate the objective as

\begin{equation}
    \mathcal{L}(\theta)
    =
    \mathbb{E}_{\bm{x}}
    \left[
        - \sum_{t=1}^{T}
        \log p_{\theta}(x_t \mid x_{<t}, \bm{z})
    \right]
    +
    \lambda \, \|\theta\|^2_2,
    \label{eq:latent_obj}
\end{equation}
where $x_{<t}$ denotes the preceding token sequence, $\lambda$ is a regularization coefficient, and $\|\theta\|_2$ is the $\ell_2$ norm of model parameters.

To integrate graph-based relational information, we define an auxiliary objective

\begin{equation}
    \mathcal{L}_{\mathrm{graph}}
    =
    \left\|
        \bm{W}_{\mathrm{aug}}^{\top} \bm{x}
        -
        \hat{\bm{x}}
    \right\|_2^2,
    \label{eq:graph}
\end{equation}
where $\bm{W}_{\mathrm{aug}}$ is a learnable transformation matrix and $\hat{\bm{x}}$ is a predicted latent reconstruction.

Combining (\ref{eq:sft}) and (\ref{eq:graph}), our final objective becomes

\begin{equation}
    \mathcal{L}_{\mathrm{total}}(\theta)
    =
    \mathcal{L}_{\mathrm{SFT}}(\theta)
    +
    \alpha \, \mathcal{L}_{\mathrm{graph}},
    \label{eq:total}
\end{equation}
where $\alpha$ controls the contribution of the structural regularization term.

The model parameters are updated via gradient descent as

\begin{equation}
    \theta_{t+1}
    =
    \theta_t
    -
    \eta \,
    \nabla_{\theta} \mathcal{L}_{\mathrm{total}}(\theta_t),
\end{equation}
where $\eta$ denotes the learning rate.

\end{document}
"""

# ② texファイルとして保存
with open("generated.tex", "w", encoding="utf-8") as f:
    f.write(latex_code)

# ③ LaTeXコマンドが使えるようにTexLiveをインストール
!apt-get update -qq
!apt-get install -y texlive-latex-base texlive-latex-extra texlive-fonts-recommended > /dev/null

# ④ pdflatexでPDFに変換
!pdflatex -interaction=nonstopmode generated.tex > /dev/null

# ⑤ 出力確認
print("PDF生成完了: generated.pdf")
!ls -lh generated.pdf


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
PDF生成完了: generated.pdf
-rw-r--r-- 1 root root 135K Nov 29 11:52 generated.pdf
